In [29]:
import os
import numpy as np
import pandas as pd
from MCForecastTools import MCSimulation
from pathlib import Path
%matplotlib inline
import alpaca_trade_api as tradeapi

In [30]:
aapl_path = Path('Data/AAPL_daily_price.csv')
amazon_path = Path('Data/AMZN_daily_price.csv')
dowj_path = Path('Data/DOWJ_daily_price.csv')
fb_path = Path('Data/FB_daily_price.csv')
msft_path = Path('Data/MSFT_daily_price.csv')
nasdaq_path = Path('Data/NASDAQ_daily_price.csv')
nflx_path = Path('Data/NFLX_daily_price.csv')
sp500_path = Path('Data/SP500_daily_price.csv')
tsla_path = Path('Data/TSLA_daily_price.csv')
vix_path = Path('Data/VIX_daily_price.csv')
wmt_path = Path('Data/WMT_daily_price.csv')

In [31]:
AAPL_csv = pd.read_csv(aapl_path, index_col= 'Date',infer_datetime_format = True, parse_dates = True)
AMAZ_csv = pd.read_csv(amazon_path, index_col='Date',infer_datetime_format = True, parse_dates = True)
DOWJ_csv = pd.read_csv(dowj_path, index_col='Date',infer_datetime_format = True, parse_dates = True)
FB_csv = pd.read_csv(fb_path, index_col='Date',infer_datetime_format = True, parse_dates = True)
MSFT_csv = pd.read_csv(msft_path, index_col='Date',infer_datetime_format = True, parse_dates = True)
NASDAQ_csv = pd.read_csv(nasdaq_path, index_col='Date',infer_datetime_format = True, parse_dates = True)
NFLX_csv = pd.read_csv(nflx_path, index_col='Date',infer_datetime_format = True, parse_dates = True)
SP500_csv = pd.read_csv(sp500_path, index_col='Date',infer_datetime_format = True, parse_dates = True)
TSLA_csv = pd.read_csv(tsla_path, index_col='Date',infer_datetime_format = True, parse_dates = True)
VIX_csv = pd.read_csv(vix_path, index_col='Date',infer_datetime_format = True, parse_dates = True)
WMT_csv = pd.read_csv(wmt_path, index_col='Date',infer_datetime_format = True, parse_dates = True)

#rename columns
AAPL_csv = AAPL_csv.rename(columns={'Close':'AAPL'})
AMAZ_csv = AMAZ_csv.rename(columns={'Close':'AMZN'})
DOWJ_csv = DOWJ_csv.rename(columns={'Close':'DOWJ'})
FB_csv = FB_csv.rename(columns={'Close':'FB'})
MSFT_csv = MSFT_csv.rename(columns={'Close':'MSFT'})
NASDAQ_csv = NASDAQ_csv.rename(columns={'Close':'NASDAQ'})
NFLX_csv = NFLX_csv.rename(columns={'Close':'NFLX'})
SP500_csv = SP500_csv.rename(columns={'Close':'S&P500'})
TSLA_csv = TSLA_csv.rename(columns={'Close':'TSLA'})
VIX_csv = VIX_csv.rename(columns={'Close':'VIX'})
WMT_csv = WMT_csv.rename(columns={'Close':'WMT'})

NASDAQ_csv.index = pd.to_datetime(NASDAQ_csv.index, format = '%Y-%m-%d').strftime('%Y-%m-%d')
VIX_csv.index = pd.to_datetime(VIX_csv.index, format = '%Y-%m-%d').strftime('%Y-%m-%d')
SP500_csv.index = pd.to_datetime(SP500_csv.index, format = '%Y-%m-%d').strftime('%Y-%m-%d')
DOWJ_csv.index = pd.to_datetime(DOWJ_csv.index, format = '%Y-%m-%d').strftime('%Y-%m-%d')



In [32]:
stocks = pd.concat([AAPL_csv,AMAZ_csv,FB_csv,MSFT_csv,NFLX_csv,TSLA_csv,WMT_csv], join='inner', axis='columns')
stocks = stocks.drop(columns='Symbol')

stocks.index = pd.to_datetime(stocks.index, format = '%Y-%m-%d').strftime('%Y-%m-%d')
stocks.head()

,AAPL,AMZN,FB,MSFT,NFLX,TSLA,WMT
Date,,,,,,,
2015-01-02,27.33,308.52,78.45,46.76,49.85,43.86,85.90
2015-01-05,26.56,302.19,77.19,46.33,47.31,42.02,85.65
2015-01-06,26.57,295.29,76.15,45.65,46.50,42.26,86.31
2015-01-07,26.94,298.42,76.15,46.23,46.74,42.19,88.60
2015-01-08,27.97,300.46,78.18,47.59,47.78,42.12,90.47


In [33]:
indexes = pd.concat([NASDAQ_csv,DOWJ_csv,SP500_csv,VIX_csv],join='inner',axis='columns')
indexes = indexes.drop(columns='Symbol')
indexes.head()


,NASDAQ,DOWJ,S&P500,VIX
Date,,,,
2015-01-02,4726.81,17832.99,2058.20,17.79
2015-01-05,4652.57,17501.65,2020.58,19.92
2015-01-06,4592.74,17371.64,2002.61,21.12
2015-01-07,4650.47,17584.52,2025.90,19.31
2015-01-08,4736.19,17907.87,2062.14,17.01


In [34]:


daily_change_indexes = indexes.pct_change()

##remove all null values#

daily_change_indexes = daily_change_indexes.dropna()
daily_change_indexes.head()


,NASDAQ,DOWJ,S&P500,VIX
Date,,,,
2015-01-05,-0.015706,-0.018580,-0.018278,0.119730
2015-01-06,-0.012860,-0.007428,-0.008893,0.060241
2015-01-07,0.012570,0.012254,0.011630,-0.085701
2015-01-08,0.018433,0.018388,0.017888,-0.119109
2015-01-09,-0.006782,-0.009521,-0.008404,0.031746


In [35]:
daily_change_stocks = stocks.pct_change()
daily_change_stocks = daily_change_stocks.dropna()
daily_change_stocks.head()


,AAPL,AMZN,FB,MSFT,NFLX,TSLA,WMT
Date,,,,,,,
2015-01-05,-0.028174,-0.020517,-0.016061,-0.009196,-0.050953,-0.041952,-0.002910
2015-01-06,0.000377,-0.022833,-0.013473,-0.014677,-0.017121,0.005712,0.007706
2015-01-07,0.013925,0.010600,0.000000,0.012705,0.005161,-0.001656,0.026532
2015-01-08,0.038233,0.006836,0.026658,0.029418,0.022251,-0.001659,0.021106
2015-01-09,0.001073,-0.011749,-0.005628,-0.008405,-0.015488,-0.018756,-0.012380


In [39]:
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

api = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version = "v2"
)


ValueError: ('Key ID must be given to access Alpaca trade API', ' (env: APCA_API_KEY_ID)')

In [37]:
num_sims = 500

# Configure a Monte Carlo simulation to forecast one year daily returns
MC_AAPL = MCSimulation(
    portfolio_data = stocks,
    num_simulation = num_sims,
    num_trading_days = 252
)

IndexError: Too many levels: Index has only 1 level, not 2

In [38]:
MC_AAPL.calc_cumulative_return()

NameError: name 'MC_AAPL' is not defined